# Este es el notebook principal para textos en inglés del que tomaremos el modelo y vectorizador para la app.py 
Recoge el modelo SVM tras aplicar técnicas para el desbalanceo como SMOTE y la ponderación de clases.

Usamos GridSearchCV para encontrar la mejor combinación de hiperparámetros.

Finalizamos calibrando el modelo para permitir sacar las probabilidades de que el texto sea spam o ham.

Aplicamos tecnicas de desbalanceo al SVM

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el conjunto de datos
df = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')

# Revisar las primeras filas del dataframe
print(df.head())

# Revisar la distribución de clases
print(df['label'].value_counts())

# Separar características y etiquetas
X = df['text']
y = df['label_num']  # Asegúrate de tener una columna numérica para las etiquetas


   Unnamed: 0 label                                               text  \
0         605   ham  enron methanol ; meter # : 988291\r\nthis is a...   
1        2349   ham  hpl nom for january 9 , 2001\r\n( see attached...   
2        3624   ham  neon retreat\r\nho ho ho , we ' re around to t...   
3        4685  spam  photoshop , windows , office . cheap . main tr...   
4        2030   ham  re : indian springs\r\nthis deal is to book th...   

   label_num  
0          0  
1          0  
2          0  
3          1  
4          0  
label
ham     3531
spam    1462
Name: count, dtype: int64


In [2]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Transformación de Texto a Características

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Ajustar y transformar los datos de entrenamiento
X_train_vec = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_vec = vectorizer.transform(X_test)


# Manejo del Desbalanceo
a. Reamostrado

Sobremuestreo de la Clase Minoritaria

In [4]:
from imblearn.over_sampling import SMOTE

# Crear el objeto SMOTE
smote = SMOTE(random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Verificar el nuevo conteo de clases
print(pd.Series(y_train_resampled).value_counts())


label_num
1    2799
0    2799
Name: count, dtype: int64


b. Ponderación de Clases en el Modelo SVM

En lugar de o además de reamostrar, puedes ajustar los pesos de las clases en el modelo SVM.

In [5]:
from sklearn.svm import SVC

# Ajustar los pesos de las clases
class_weights = {0: 1, 1: 3}  # Ajusta los pesos según el desbalanceo observado




3. Entrenamiento del Modelo SVM
Entrenamiento con Datos Balanceados

In [6]:
# Crear el modelo SVM
model_svm = SVC(class_weight=class_weights, kernel='linear', random_state=42)

# Entrenar el modelo con Datos Balanceados
model_svm.fit(X_train_resampled, y_train_resampled)


SVC(class_weight={0: 1, 1: 3}, kernel='linear', random_state=42)

Predicción y Evaluación

In [7]:
from sklearn.metrics import classification_report, confusion_matrix

# Predecir en el conjunto de prueba
y_test_pred = model_svm.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print("Reporte de Clasificación:")
print(classification_report(y_test, y_test_pred))

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_test_pred))

print(f"Exactitud en el conjunto de prueba: {model_svm.score(X_test_vec, y_test)}")


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       732
           1       0.93      0.97      0.95       267

    accuracy                           0.97       999
   macro avg       0.96      0.97      0.97       999
weighted avg       0.97      0.97      0.97       999

Matriz de Confusión:
[[713  19]
 [  7 260]]
Exactitud en el conjunto de prueba: 0.973973973973974


4. Ajustes y Validación Cruzada
Ajustar Parámetros del Modelo
Puedes ajustar los parámetros del modelo SVM utilizando GridSearchCV para encontrar la mejor combinación de hiperparámetros.

In [8]:
from sklearn.model_selection import GridSearchCV

# Definir los parámetros a ajustar
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'class_weight': [None, 'balanced']  # También puedes ajustar los pesos manualmente aquí
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, n_jobs=-1)

# Ajustar el modelo con los datos balanceados
grid_search.fit(X_train_resampled, y_train_resampled)

# Mejor combinación de parámetros
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores parámetros
best_model = grid_search.best_estimator_
y_test_pred_best = best_model.predict(X_test_vec)

print("Reporte de Clasificación con GridSearchCV:")
print(classification_report(y_test, y_test_pred_best))

print("Matriz de Confusión con GridSearchCV:")
print(confusion_matrix(y_test, y_test_pred_best))


Mejores parámetros encontrados:
{'C': 10, 'class_weight': None, 'kernel': 'rbf'}
Reporte de Clasificación con GridSearchCV:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       732
           1       0.95      0.97      0.96       267

    accuracy                           0.98       999
   macro avg       0.97      0.98      0.97       999
weighted avg       0.98      0.98      0.98       999

Matriz de Confusión con GridSearchCV:
[[719  13]
 [  7 260]]


Los resultados del mejor modelo encontrado usando GridSearchCV son bastante buenos y muestran que el modelo está funcionando muy bien. Aquí hay un análisis detallado de los resultados:

### Parámetros del Mejor Modelo
- **C**: 10
- **class_weight**: None
- **kernel**: rbf

### Reporte de Clasificación
El reporte de clasificación proporciona métricas clave sobre el rendimiento del modelo para cada clase (0 y 1). Aquí está el desglose:

#### Clase 0:
- **Precision (Precisión)**: 0.99
- **Recall (Sensibilidad)**: 0.98
- **F1-score**: 0.99
- **Support (Soporte)**: 732

#### Clase 1:
- **Precision (Precisión)**: 0.95
- **Recall (Sensibilidad)**: 0.97
- **F1-score**: 0.96
- **Support (Soporte)**: 267

#### Totales:
- **Accuracy (Precisión global)**: 0.98
- **Macro avg**:
  - Precision: 0.97
  - Recall: 0.98
  - F1-score: 0.97
- **Weighted avg**:
  - Precision: 0.98
  - Recall: 0.98
  - F1-score: 0.98

### Interpretación de las Métricas
1. **Precision (Precisión)**: Indica la proporción de verdaderos positivos sobre todos los positivos predichos. Una precisión de 0.99 para la clase 0 y 0.95 para la clase 1 muestra que el modelo es muy bueno en predecir correctamente las instancias positivas de ambas clases.
  
2. **Recall (Sensibilidad)**: Indica la proporción de verdaderos positivos sobre todos los positivos reales. Un recall de 0.98 para la clase 0 y 0.97 para la clase 1 indica que el modelo es muy eficaz en identificar la mayoría de las instancias positivas de cada clase.

3. **F1-score**: Es la media armónica de la precisión y el recall. Valores altos de F1-score (0.99 para la clase 0 y 0.96 para la clase 1) indican que el modelo tiene un buen balance entre precisión y recall.

4. **Accuracy (Precisión global)**: Con un valor de 0.98, indica que el modelo clasifica correctamente el 98% de todas las instancias.

5. **Macro avg** y **Weighted avg**: Estas métricas promedian las métricas individuales de cada clase. La media ponderada considera el soporte (número de instancias en cada clase), mientras que la media macro no. Ambas muestran valores altos, lo que sugiere que el modelo tiene un rendimiento equilibrado en ambas clases.

### Matriz de Confusión
La matriz de confusión ofrece una visión detallada de los verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos:
- [[719, 13],
-  [7, 260]]

#### Interpretación: ham = 0 y spam = 1
- **719**: Verdaderos negativos (clase 0 correctamente predicha)
- **13**: Falsos positivos (clase 0 predicha incorrectamente como clase 1)
- **7**: Falsos negativos (clase 1 predicha incorrectamente como clase 0)
- **260**: Verdaderos positivos (clase 1 correctamente predicha)

Estos resultados muestran que el modelo comete muy pocos errores en ambas clases, con solo 20 errores en un total de 999 instancias.

### Conclusión
El modelo entrenado con los parámetros {'C': 10, 'class_weight': None, 'kernel': 'rbf'} demuestra un rendimiento excelente con una precisión, sensibilidad y F1-score muy altos en ambas clases, y una precisión global del 98%. La matriz de confusión respalda estas métricas, mostrando un número muy bajo de falsos positivos y falsos negativos. Esto sugiere que el modelo es altamente fiable para clasificar correctamente las instancias de ambas clases en el conjunto de datos dado.

## Validación Cruzada
Finalmente, validamos el modelo usando validación cruzada para asegurarte de que el rendimiento es consistente.

In [9]:
from sklearn.model_selection import cross_val_score

# Validación cruzada
cv_scores = cross_val_score(best_model, X_train_resampled, y_train_resampled, cv=5)
print(f"Exactitud promedio en validación cruzada: {cv_scores.mean()}")


Exactitud promedio en validación cruzada: 0.9894599770202988


In [10]:
import pickle

pickle_file = "../modelos_y_vectorizadores_en/best_model_svm.pkl"
with open(pickle_file, 'wb') as file:
    pickle.dump(best_model, file)
print(f"Modelo guardado como {pickle_file}")


Modelo guardado como best_model_svm.pkl


In [12]:
import joblib
# Guardar el vectorizador ajustado
joblib.dump(vectorizer, '../modelos_y_vectorizadores_en/vectorizador_en.pkl')

['vectorizador_en.pkl']

El modelo SVM necesita de calibrado para poder obtener las probabilidades de spam y ham en la app.py

In [13]:
from sklearn.calibration import CalibratedClassifierCV
# Calibrar el modelo
calibrated_svm = CalibratedClassifierCV(best_model, method='sigmoid')
calibrated_svm.fit(X_train_resampled, y_train_resampled)

# Guardar el modelo calibrado
with open('../modelos_y_vectorizadores_en/calibrated_svm_model.pkl', 'wb') as f:
    pickle.dump(calibrated_svm, f)

print("Modelo calibrado y guardado exitosamente.")

Modelo calibrado y guardado exitosamente.
